# Build the logbook for Hologram with empty or filters

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : October 30th 2022
- last update : November 8th 2022


Works directory per directory (day by day)

CCIN2P3:

- works with kernels **anaconda3_auxtel** (with libradtran) at CC
- works with kernel **python 3** locally

In [1]:
import sys
sys.path

['/pbs/throng/lsst/users/dagoret/desc/AuxTelComm/notebookccdm_gen3/runspectractor_standalone/2022_11',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python38.zip',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/pbs/home/d/dagoret/.local/lib/python3.8/site-packages',
 '/pbs/throng/lsst/users/dagoret/desc/firecrown',
 '/pbs/throng/lsst/users/dagoret/desc/augur',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/Deprecated-1.2.11-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/docopt-0.6.2-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/mimeparse-0.1.3-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/rail-0.1.dev0-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/a

In [2]:
! ls /sps/lsst/groups/auxtel/data/2022/BG40~holo4_003/20220629

exposure_2022062900054_postisrccd.fits	exposure_2022062900325_postisrccd.fits
exposure_2022062900055_postisrccd.fits	exposure_2022062900327_postisrccd.fits
exposure_2022062900056_postisrccd.fits	exposure_2022062900328_postisrccd.fits
exposure_2022062900084_postisrccd.fits	exposure_2022062900359_postisrccd.fits
exposure_2022062900110_postisrccd.fits	exposure_2022062900360_postisrccd.fits
exposure_2022062900113_postisrccd.fits	exposure_2022062900362_postisrccd.fits
exposure_2022062900137_postisrccd.fits	exposure_2022062900388_postisrccd.fits
exposure_2022062900138_postisrccd.fits	exposure_2022062900389_postisrccd.fits
exposure_2022062900139_postisrccd.fits	exposure_2022062900414_postisrccd.fits
exposure_2022062900164_postisrccd.fits	exposure_2022062900415_postisrccd.fits
exposure_2022062900165_postisrccd.fits	exposure_2022062900416_postisrccd.fits
exposure_2022062900166_postisrccd.fits	exposure_2022062900446_postisrccd.fits
exposure_2022062900201_postisrccd.fits	exposure_2022062900449_po

# Select the combination DATE, FILTER, DISPERSER

In [3]:
DATE = '20211103'
filterdispersernames = ["empty~holo4_003","BG40~holo4_003","FELH0600~holo4_003","SDSSg~holo4_003",\
                        "empty~ronchi170lpmm","BG40~ronchi170lpmm","FELH0600~ronchi170lpmm","SDSSg~ronchi170lpmm",\
                       ]
filterdispersername = filterdispersernames[0]
version="v5.0"  # new orientation

# If position need to be specified add it in this dictionnary

In [4]:
# reconstructed / good /xpos /ypos

if version == "v5.0":
    info={\

          # with red filter
          'exposure_2022031600351_postisrccd.fits':(1,1,300.,1700),\
         }
        

# Imports

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [6]:
from astropy.io import fits

In [7]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

In [8]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [9]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [10]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [11]:
dir_logbooks="logbooks"

In [12]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [13]:
home=os.getenv("HOME")

In [14]:
image_path=filterdispersername
logbook_csv=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_"+ DATE +"_" + version + ".csv")
logbook_xlsx=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_" +DATE +"_" + version + ".xlsx")

# Path

In [15]:
HOSTCC=True
LAPTOP=False

In [16]:
# Set path depending on which computer running (according HOSTCC)
if HOSTCC:
    path_auxtel="/sps/lsst/groups/auxtel"
    path_spectractor=os.path.join(path_auxtel,"softs/github/desc/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    path_images=os.path.join(path_auxtel,"data/2022/"+filterdispersername+"/"+DATE)

else:
    if not LAPTOP:
        path_auxtel="/Users/sylvie/DATA/AuxTelData2022"
        path_spectractor=os.path.join(path_auxtel,"/Users/sylvie/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2022/"+filterdispersername+"/"+DATE)
        
    else:
        path_auxtel="/Users/dagoret/DATA/AuxTelData2022"
        path_spectractor=os.path.join(path_auxtel,"/Users/dagoret/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2022/"+filterdispersername+"/"+DATE)
        

# Check the images exists

In [17]:
dir_images=path_images

In [18]:
all_files=os.listdir(dir_images)

In [19]:
all_files

['exposure_2021110300691_postisrccd.fits',
 'exposure_2021110300501_postisrccd.fits',
 'exposure_2021110300562_postisrccd.fits',
 'exposure_2021110300537_postisrccd.fits',
 'exposure_2021110300733_postisrccd.fits',
 'exposure_2021110300709_postisrccd.fits',
 'exposure_2021110300525_postisrccd.fits',
 'exposure_2021110300697_postisrccd.fits',
 'exposure_2021110300715_postisrccd.fits',
 'exposure_2021110300580_postisrccd.fits',
 'exposure_2021110300513_postisrccd.fits',
 'exposure_2021110300592_postisrccd.fits',
 'exposure_2021110300610_postisrccd.fits',
 'exposure_2021110300703_postisrccd.fits',
 'exposure_2021110300465_postisrccd.fits',
 'exposure_2021110300574_postisrccd.fits',
 'exposure_2021110300459_postisrccd.fits',
 'exposure_2021110300586_postisrccd.fits',
 'exposure_2021110300489_postisrccd.fits',
 'exposure_2021110300556_postisrccd.fits',
 'exposure_2021110300483_postisrccd.fits',
 'exposure_2021110300543_postisrccd.fits',
 'exposure_2021110300519_postisrccd.fits',
 'exposure_

In [20]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [21]:
all_selected_files=np.array(all_selected_files)

### Sort files

In [22]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('exposure.*_(.*)_postisrccd[.]fits$',filename)
    
    return m[0]

In [23]:
file_tag_forsorting('exposure_2022031700310_postisrccd.fits')

'2022031700310'

In [24]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['2021110300691', '2021110300501', '2021110300562', '2021110300537',
       '2021110300733', '2021110300709', '2021110300525', '2021110300697',
       '2021110300715', '2021110300580', '2021110300513', '2021110300592',
       '2021110300610', '2021110300703', '2021110300465', '2021110300574',
       '2021110300459', '2021110300586', '2021110300489', '2021110300556',
       '2021110300483', '2021110300543', '2021110300519', '2021110300568',
       '2021110300604', '2021110300471', '2021110300598', '2021110300507',
       '2021110300477', '2021110300531', '2021110300721', '2021110300495',
       '2021110300727'], dtype='<U13')

In [25]:
indexes=np.argsort(all_tags)

In [26]:
sorted_files=all_selected_files[indexes]

In [27]:
sorted_files

array(['exposure_2021110300459_postisrccd.fits',
       'exposure_2021110300465_postisrccd.fits',
       'exposure_2021110300471_postisrccd.fits',
       'exposure_2021110300477_postisrccd.fits',
       'exposure_2021110300483_postisrccd.fits',
       'exposure_2021110300489_postisrccd.fits',
       'exposure_2021110300495_postisrccd.fits',
       'exposure_2021110300501_postisrccd.fits',
       'exposure_2021110300507_postisrccd.fits',
       'exposure_2021110300513_postisrccd.fits',
       'exposure_2021110300519_postisrccd.fits',
       'exposure_2021110300525_postisrccd.fits',
       'exposure_2021110300531_postisrccd.fits',
       'exposure_2021110300537_postisrccd.fits',
       'exposure_2021110300543_postisrccd.fits',
       'exposure_2021110300556_postisrccd.fits',
       'exposure_2021110300562_postisrccd.fits',
       'exposure_2021110300568_postisrccd.fits',
       'exposure_2021110300574_postisrccd.fits',
       'exposure_2021110300580_postisrccd.fits',
       'exposure_202

In [28]:
N=len(sorted_files)

# Build the logbook from the headers

In [29]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=hdr0['OBJECT']
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [30]:
df=pd.DataFrame()

In [31]:
len(all_airmass)

33

In [32]:
N=len(all_indexes)

## Fill with container info

In [33]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [34]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [35]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [36]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [37]:
pd.set_option('display.max_rows', None)

In [38]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2021-11-04T05:23:15.100,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2021110300459_postisrccd.fits,459,HD 38666,empty_1,holo4_003,1.134752,30.0,...,7.3,32.0,5.1,NaN,NaN,NaN,NaN,,NaN,NaN
1,1,2021-11-04T05:27:18.411,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2021110300465_postisrccd.fits,465,HD 38666,empty_1,holo4_003,1.125856,30.0,...,7.4,31.0,6.5,NaN,NaN,NaN,NaN,,NaN,NaN
2,2,2021-11-04T05:31:21.800,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2021110300471_postisrccd.fits,471,HD 38666,empty_1,holo4_003,1.117341,30.0,...,7.3,33.0,6.0,NaN,NaN,NaN,NaN,,NaN,NaN
3,3,2021-11-04T05:35:29.984,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2021110300477_postisrccd.fits,477,HD 38666,empty_1,holo4_003,1.109040,30.0,...,7.2,33.0,5.4,NaN,NaN,NaN,NaN,,NaN,NaN
4,4,2021-11-04T05:39:32.920,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2021110300483_postisrccd.fits,483,HD 38666,empty_1,holo4_003,1.101273,30.0,...,7.2,33.0,5.8,NaN,NaN,NaN,NaN,,NaN,NaN
5,5,2021-11-04T05:43:36.197,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2021110300489_postisrccd.fits,489,HD 38666,empty_1,holo4_003,1.093841,30.0,...,7.2,31.0,6.7,NaN,NaN,NaN,NaN,,NaN,NaN
6,6,2021-11-04T05:47:45.320,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2021110300495_postisrccd.fits,495,HD 38666,empty_1,holo4_003,1.086590,30.0,...,7.4,29.0,6.5,NaN,NaN,NaN,NaN,,NaN,NaN
7,7,2021-11-04T05:51:48.500,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2021110300501_postisrccd.fits,501,HD 38666,empty_1,holo4_003,1.079841,30.0,...,7.5,27.0,5.3,NaN,NaN,NaN,NaN,,NaN,NaN
8,8,2021-11-04T05:55:51.931,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2021110300507_postisrccd.fits,507,HD 38666,empty_1,holo4_003,1.073401,30.0,...,7.4,31.0,6.1,NaN,NaN,NaN,NaN,,NaN,NaN
9,9,2021-11-04T06:00:24.076,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2021110300513_postisrccd.fits,513,HD 38666,empty_1,holo4_003,1.066577,30.0,...,7.4,31.0,6.9,NaN,NaN,NaN,NaN,,NaN,NaN


# Make logbook


In [39]:
N=len(df)
N

33

In [40]:
if N>0:
    df.to_excel(logbook_xlsx)
    df.to_csv(logbook_csv)

In [41]:
logbook_csv

'logbooks/auxtellogbook_empty~holo4_003_20211103_v5.0.csv'